# 프롬프팅 지침
이 수업에서는 대형 언어 모델을 위한 효과적인 프롬프트를 작성하기 위해 두 가지 프롬프트 원칙과 관련된 작성 방법을 연습합니다.

## 설정
#### API 키 및 관련 Python 라이브러리를 로드합니다.

이 과정에서는 OpenAI API 키를 로드하는 코드를 제공합니다.

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

#### 도우미 함수
이 과정에서는 OpenAI의 gpt-3.5-turbo 모델과 [chat completions endpoint](https://platform.openai.com/docs/guides/chat) 를 사용합니다. 

이 도우미 함수를 사용하면, 프롬프트를 더욱 편리하게 사용하고, 쉽게 생성된 출력을 볼 수 있습니다:

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## 프롬프팅 원칙
- **원칙 1: 명확하고 구체적인 지침을 작성합니다**
- **원칙 2: 모델에게 "생각"할 시간을 줍니다**

### 방법

#### 방법 1. 입력값 부분을 명확히 구분하기 위해 구분 문자(delimiter)를 사용한다.
- 구분 문자 예시: ```, """, <>, <tag> </tag>, :

In [ ]:
text = f"""
가능한 한 명확하고 구체적인 지침을 제공하여 모델이 수행하기를 원하는 작업을 표현해야 합니다. 그러면 모델이 원하는 내용을 출력하도록 유도할 수 있고, 부적절한 응답을 받을 가능성을 줄일 수 있습니다. 프롬프트를 명확하게 쓰는 것과 프롬프트를 짧게 쓰는 것은 다릅니다. 대부분의 경우 긴 프롬프트가 모델에게 더 명확한 정보와 맥락을 제공하므로 보다 상세하고 관련성 있는 결과물로 이어질 수 있습니다.
"""
prompt = f"""
삼중 백틱으로 구분된 텍스트를 한 문장으로 요약하시오.
```{text}```
"""
response = get_completion(prompt)
print(response)

#### 방법 2. 결과물 형식을 알려준다.
- 결과물 형식 예시: JSON, HTML

In [ ]:
prompt = f"""
저자 및 장르와 함께 세 개의 구성된 책 제목 목록을 생성하시오.
다음 키를 사용하여 JSON 형식으로 제공하시오:
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

#### 방법 3. 모델에게 조건 충족 여부를 확인하라고 한다.

In [ ]:
text_1 = f"""
차 한 잔을 만드는 것은 쉽습니다! 먼저, 물을 끓여야 합니다. 그럴 때는 컵을 들고 티백을 넣어주세요. 물이 충분히 뜨거우면 티백 위에 붓기만 하면 됩니다. 차가 끓을 수 있도록 잠시 놔두세요. 몇 분 후에 티백을 꺼내세요. 원한다면 맛을 위해 설탕이나 우유를 첨가할 수 있습니다. 그게 다에요! 당신은 맛있는 차 한 잔을 즐길 수 있습니다.
"""
prompt = f"""
세 개의 따옴표로 구분된 텍스트가 제공됩니다.
작업 단계가 포함된 경우 다음 형식으로 해당 작업 내용을 다시 작성하시오:

Step 1 - ...
Step 2 - …
…
Step N - …

텍스트에 작업 단계가 없다면, \"작업 순서를 제공할 수 없습니다.\"라고 하시오

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

In [ ]:
text_2 = f"""
오늘은 태양이 밝게 빛나고, 새들이 지저귀고 있다. 공원으로 산책을 가기에 좋은 날씨입니다. 꽃이 피고, 나무들이 산들바람에 살랑살랑 흔들리고 있다. 사람들은 밖에서 돌아다니며 멋진 날씨를 즐기고 있다. 어떤 사람들은 소풍을 가고 있는 반면, 다른 사람들은 게임을 하거나 잔디 위에서 휴식을 취하고 있다. 야외에서 시간을 보내고 자연의 아름다움을 감상하기에 완벽한 날입니다.
"""
prompt = f"""
세 개의 따옴표로 구분된 텍스트가 제공됩니다. 
일련의 지시가 포함되어 있다면, 해당 지침을 다음 형식으로 다시 작성합니다:

Step 1 - ...
Step 2 - …
…
Step N - …

텍스트에 일련의 지침이 포함되어 있지 않으면 \"단계가 제공되지 않습니다.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

#### 방법 4. 모범 사례를 알려준다. (”Few-shot” prompting)

In [ ]:
prompt = f"""
당신의 임무는 일관된 문체로 대답하는 것이다.

<자녀>: 인내심에 대해 가르쳐 주세요.

<조부모>: 가장 깊은 계곡을 깎는 강은 수수한 샘에서 시작되고, 가장 웅장한 교향곡은 하나의 음표에서 비롯되며, 가장 화려한 태피스트리는 하나의 실에서 시작된다.

<자녀>: 회복력에 대해 가르쳐 주세요.
"""
response = get_completion(prompt)
print(response)

### 원칙 2: AI 모델이 생각할 시간을 준다.

#### 방법 1. 요청할 작업의 절차를 세분화한다.

In [ ]:
text = f"""
매력적인 마을에서, 남매 잭과 질은 언덕 꼭대기 우물에서 물을 길어오기 위해 탐험을 떠났다. 그들이 즐겁게 노래를 부르며 오는데, 순간 불행이 닥쳤다. 잭은 돌에 걸려 언덕 아래로 굴러 떨어졌고, 질은 그 뒤를 따랐다. 비록 만신창이가 되었지만, 그 두 사람은 집으로 돌아와 위로의 포옹을 했다. 불행에도 불구하고, 그들의 모험심은 흔들리지 않았고, 그들은 기쁨으로 탐험을 계속했다.
"""
# example 1
prompt_1 = f"""
다음 작업을 수행합니다: 
1 - 다음 텍스트를 1문장의 삼중 백틱으로 구분하여 요약합니다.
2 - 요약을 프랑스어로 번역합니다.
3 - 프랑스어 요약에 각 이름을 나열합니다.
4 - 다음을 포함하는 json 객체를 출력합니다 
다음 키: french_names, num_names.

답을 줄 바꿈으로 구분합니다.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

#### 지정된 형식 출력 요청

In [ ]:
prompt_2 = f"""
작업은 다음 작업을 수행하는 것입니다: 
1 - 다음 텍스트를 요약합니다 
  1문장의 <>.
2 - 요약을 프랑스어로 번역합니다.
3 - 프랑스어 요약에 각 이름을 나열합니다.
4 - 다음을 포함하는 json 객체를 출력합니다 
  다음 키: french_names, num_names.

다음 형식을 사용합니다:
텍스트: <요약할 텍스트>
요약: <요약>
번역 : <요약 번역>
이름: <이탈리아어 요약에 있는 이름 목록>
출력 JSON: <요약과 num_name이 있는 json>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)

#### 방법 2. 바로 정답을 구하지 말고, 모델이 본인만의 답을 먼저 만들게 한다.

In [ ]:
prompt = f"""
학생의 솔루션이 올바른지 여부를 결정합니다.

질문:.
태양광 발전 시설을 짓고 있는데 재정 문제를 해결하는 데 도움이 필요합니다.
- 땅값은 평방 피트당 100달러다
- 나는 평방 피트당 250달러에 태양 전지판을 살 수 있다
- 나는 연간 10만 달러의 고정 비용과 평방 피트 당 10달러의 추가 비용이 드는 유지 보수 계약을 협상했다. 평방 피트의 수에 대한 함수로서 운영 첫 해의 총 비용은 얼마입니까?

학생 답:
x를 평방 피트 단위의 설비 크기로 하자.
비용:
1. 지가: 100x
2. 태양 전지판 가격: 250x
3. 유지 보수비 : 100,000 + 100x
   총 비용: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

#### 사실, 학생의 답은 틀렸습니다.
#### 우리는 모델에게 답을 먼저 제시하도록 지시함으로써 이 문제를 해결할 수 있습니다.

In [ ]:
prompt = f"""
당신의 임무는 학생의 답이 올바른지 여부를 판단하는 것입니다.

다음 지시에 따라 문제를 해결하시오:
1. 먼저, 문제에 대한 AI의 답을 적으시오.
2. AI의 답을 학생의 답과 비교하시오.
3. 학생의 답이 올바른지 평가하시오.

주의사항
- 직접 문제를 해결하기 전에는 학생의 답이 맞는지 결정하지 마시오.
- 아래 형식에 맞춰 결과를 작성하시오
    - AI의 답:
    ```
    AI의 답을 적으시오.
    ```

    - 학생의 답:
    ```
    여기에 학생의 답을 적으시오.
    ```

    - 학생의 답이 방금 계산된 AI의 답과 동일합니까?:
    ```
    예/아니오
    ```

    - 채점 결과:
    ```
    정답/오답
    ```
------------------

- 질문:
    ```
        태양광 발전 시설을 짓고 있는데 재정 문제를 해결하는 데 도움이 필요합니다. 
        - 땅값은 평방피트당 100달러다
        - 나는 평방피트당 250달러에 태양 전지판을 살 수 있다
        - 나는 연간 10만 달러의 고정 비용과 평방피트당 10달러의 추가 비용이 드는 유지보수 계약을 협상했다. 평방 피트의 수에 대한 함수로서 운영 첫 해의 총 비용은 얼마입니까?
    ``` 
- 학생의 답:
    ```
        x를 평방 피트 단위의 설비 크기로 하자.
        비용:
        1. 지가: 100x
        2. 태양 전지판 가격: 250x
        3. 유지 보수비 : 100,000 + 100x
        총 비용: 100x + 250x + 100,000 + 100x = 450x + 100,000
    ```
"""
response = get_completion(prompt)
print(response)

## 모델 한계: 허구
- "보이"는 실제 존재하는 회사고, 제품은 실제로 존재하지 않습니다.

In [ ]:
prompt = f"""
보이의 에어로글라이드 울트라슬림 스마트 칫솔에 대해 알려줘
"""
response = get_completion(prompt)
print(response)

## 스스로 실험해 보세요!

#### 강의실 외부에서 OpenAI API 사용에 대한 참고 사항

OpenAI Python 라이브러리를 설치하려면:
```
!pip install openai
```

라이브러리는 [website](https://platform.openai.com/account/api-keys)에서 사용할 수 있는 계정의 비밀 키로 구성해야 합니다.

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

#### 백슬래시(\)에 대한 참고 사항
- 이 과정에서 우리는 줄 바꿈 문자 \n을 삽입하지 않고 텍스트를 화면에 맞게 만들기 위해 백슬래시 \"를 사용하고 있습니다.
- GPT-3는 당신이 새로운 줄 문자를 넣든 안 넣든 별로 영향을 받지 않습니다.  그러나 일반적으로 LLM을 사용할 때는 프롬프트의 새 줄 문자가 모델의 성능에 영향을 미치는지 여부를 고려할 수 있습니다.